In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
import joblib


# 로드
scaler = joblib.load("./my_scaler.1line.pkl")
model = load_model("./my_dl_model.1line.keras")
exit_df = pd.read_csv(
    "../../data/과정/3.빠른출입구/전철역 가까운 탑승구_가공_최종.csv", encoding="euc-kr"
)

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
def parse_exit_weights(exit_str, num_cars=10):
    weights = np.ones(num_cars)
    if pd.isna(exit_str):
        return weights
    for val in str(exit_str).split(","):
        val = val.strip()
        if "-" in val:
            try:
                car = int(val.split("-")[0])
                if 1 <= car <= num_cars:
                    weights[car - 1] += 1
            except:
                continue
    return weights

In [11]:
def get_exit_weights(exit_df, station_name, direction, num_cars=10):
    row = exit_df[exit_df["역명"] == station_name]
    if row.empty:
        print(f"[⚠️] '{station_name}' 역 없음")
        return np.ones(num_cars)

    col = "가까운 출구" if direction == "상행" else "가까운 출구.1"

    if col not in row.columns:
        print(f"[⚠️] 출구 칼럼 없음: {col}")
        return np.ones(num_cars)

    return parse_exit_weights(row.iloc[0][col], num_cars)

In [12]:
station_name = input("🚉 역명 입력: ").strip()
hour = int(input("🕐 시간 입력 (0~23): ").strip())
weekday = int(input("📅 평일=0, 주말=1: ").strip())
direction = input("↕️ 방향 입력 (상행/하행): ").strip()

# 역번호 매핑 (출입구 CSV에서 자동 가져오기)
역번호_map = exit_df.set_index("역명")["역번호"].to_dict()
station_code = 역번호_map.get(station_name, 999)

if station_code == 999:
    print(f"[⚠️] '{station_name}' 역번호 찾을 수 없음")

In [13]:
# 1. 1차 예측에 사용될 입력 (승하차/혼잡도는 0 또는 None)
X_input = np.array(
    [
        [
            station_code,
            hour,
            weekday,
            1 if direction == "상행" else 0,
            1,
            0,  # 승하차인원 (초기값)
            0,  # 혼잡도 (초기값)
        ]
    ]
)
scaled_input = scaler.transform(X_input)
total_passenger = model.predict(scaled_input)[0, 0]

# 음수값 방지
total_passenger = max(total_passenger, 0)

# 2. 혼잡도는 산식으로 계산 (실제 정원이 필요)
train_capacity = 1600
congestion = min(total_passenger / train_capacity * 100, 200)  # 200% 상한

# 3. 승하차인원은, 총 인원과 비례해 산정하거나 데이터, 예측로직 마련 (없으면 0~평균값 활용)
#    혼잡도가 AI의 별도 출력일 경우, pred[1] 사용

# 후처리(분배)는 동일

# 4. 분배
exit_weights = get_exit_weights(exit_df, station_name, direction, num_cars)
proportions = exit_weights / exit_weights.sum()
car_passenger = total_passenger * proportions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


In [14]:
# 혼잡도 계산
train_capacity = 1600  # 열차 최대 수용 인원 (예, 열차 10칸 × 카당 160명)

congestion_rate = min((total_passenger / train_capacity) * 100, 200)  # 상한 값 설정
print(f"📊 계산된 혼잡도: {congestion_rate:.1f}%")

📊 계산된 혼잡도: 0.0%


In [15]:
num_cars = 10
exit_weights = get_exit_weights(exit_df, station_name, direction, num_cars)
proportions = exit_weights / exit_weights.sum()
car_passenger = total_passenger * proportions

In [16]:
print(f"\n🚉 {station_name} {direction} 혼잡 예측 결과:")
for i, val in enumerate(car_passenger, start=1):
    print(f"🚪 {i}번 칸 예상 탑승 인원: {val:.2f}명")


🚉 시청 상행 혼잡 예측 결과:
🚪 1번 칸 예상 탑승 인원: 0.00명
🚪 2번 칸 예상 탑승 인원: 0.00명
🚪 3번 칸 예상 탑승 인원: 0.00명
🚪 4번 칸 예상 탑승 인원: 0.00명
🚪 5번 칸 예상 탑승 인원: 0.00명
🚪 6번 칸 예상 탑승 인원: 0.00명
🚪 7번 칸 예상 탑승 인원: 0.00명
🚪 8번 칸 예상 탑승 인원: 0.00명
🚪 9번 칸 예상 탑승 인원: 0.00명
🚪 10번 칸 예상 탑승 인원: 0.00명
